<a href="https://colab.research.google.com/github/nehapokharel/practice_1/blob/main/tumor_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#importing necessary libraries
import os
import numpy as np
import pandas as pd
from google.colab import drive
import matplotlib.pyplot as plt

In [ ]:
#making content of google drive accessible
drive.mount("/content/gdrive")

Mounted at /content/gdrive


In [ ]:
!ls "gdrive/MyDrive/MRI_BRAIN_TUMOR/dataset"

no  yes


In [ ]:
classes = ["yes", "no"]
n_classes = len(classes)

In [ ]:
def load_data():
    """This function loads the data and prepares it to be 
    used by th neneural network. It consists of:
        1. loading images
        2. `normalization`. Original pixel values are in range [0, 255], 
           but neural networks usually work better if the input values are in
           range [0, 1]. """  
    x = []
    y = []
    path = "gdrive/MyDrive/MRI_BRAIN_TUMOR/dataset"
    for i, c in enumerate(classes):
        for file in os.listdir(os.path.join(path, c)):
            if file.endswith(".jpeg") or file.endswith(".jpg"):
                img = plt.imread(os.path.join(path, c, file))
                img = img / 255.
                x.append(img)
                y.append(i)
    return x, y

In [ ]:
x, y = load_data() #function call

In [ ]:
print("Length of x",len(x))
print("Length of y",len(y))

Length of x 177
Length of y 177
